# Credits

This is heavily influenced or copied from https://github.com/pytorch/tutorials

# Autograd: automatic differentiation

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our first neural network.

The `autograd` package provides automatic differentiation for all operations on Tensors.
It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

## 1. Variable

`autograd.Variable` is the central class of the package. It wraps a Tensor, and supports nearly all of operations defined on it.
Once you finish your computation you can call `.backward()` and have all the gradients computed automatically.

You can access the raw tensor through the `.data` attribute, while the gradient w.r.t. this variable is accumulated into `.grad`.

![autograd.Variable](../static_files/autograd-variable.png)

There’s one more class which is very important for autograd implementation - a `Function`.

`Variable` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each variable has a `.grad_fn` attribute that references a `Function` that has created the `Variable` (except for `Variable`s created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a `Variable`. If `Variable` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to `backward()`, however if it has more elements, you need to specify a `grad_output` argument that is a tensor of matching shape.

In [37]:
import torch
from torch.autograd import Variable

Create a variable

In [38]:
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do an operation of variable:

In [39]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward>)


`y` was created as a result of an operation, so it has a `grad_fn`.

In [40]:
print(y.grad_fn)

Do more operations on y

In [58]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([ 370414.3438,  286367.1875, 7147297.0000], grad_fn=<MulBackward>)
tensor(2601359.5000, grad_fn=<MeanBackward1>)


# Assignments

1. Create a Variable that `requires_grad` containing a tensor of size (5, 5)
2. Sum the values in the Variable

In [59]:
tes = Variable(torch.ones(5,5), requires_grad=True)
tesnxt = tes * 3 + tes

print(tesnxt)

tensor([[4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4.]], grad_fn=<ThAddBackward>)


## 2. Gradients

Let’s backprop now `out.backward()` is equivalent to doing `out.backward(torch.Tensor([1.0]))`

In [60]:
out.backward(retain_graph=True)

print gradients d(out)/dx

In [55]:
z.sum().backward()

In [56]:
print(x.grad)

tensor([ 2159012.2500,  1899268.7500, -9483344.0000])


You should have got a matrix of `4.5`. Let’s call the `out` *Variable* $o$.

We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = \frac{1}{4}3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.

Therefore, $\frac{\partial o}{\partial x_i} = \frac{6}{4}(x_i+2)$,
hence $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

You can do many crazy things with autograd!

In [46]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([  351.3850,   308.9591, -1543.5129], grad_fn=<MulBackward>)


In [47]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([ 102.4000, 1024.0000,    0.1024])


**Read later**
Documentation of `Variable` and `Function` is at http://pytorch.org/docs/autograd

# Assignments

1. Define a tensor
2. Convert the tensor to a torch.Variable that requires_grad
3. Multiply the torch.Variable with 2 and assign the result to a new python variable (i.e. `x = result`)
4. Sum the variable's elements and assign to a new python variable
5. Print the gradients of all the variables
6. Now perform a backward pass on the last variable
7. Print all gradients again
  - what did you notice?